TruncatedSVD 는 학습 결과로 S, Sigma, VT 를 얻습니다. 그런데, Scikit learn 의 TruncatedSVD 는 S * Sigma 를 input data X 의 차원축소된 값으로 return 하고, VT 를 .components_ 에 저장합니다. 때로는 S, Sigma, VT 가 각각 필요할 경우가 있습니다. Scikit-learn 의 TruncatedSVD 의 fit 함수 안을 살펴보면 이를 얻을 수 있는 코드들이 있습니다.

이번 튜토리얼에서는 scikit-learn 의 코드를 바탕으로 SVD 의 각 성분을 얻는 함수를 만듭니다.

## Data loading

In [1]:
import config
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')

soynlp=0.0.49
added lovit_textmining_dataset


## Scikit-learn SVD

In [3]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

x_ = normalize(x)
svd = TruncatedSVD(n_components=100)
y = svd.fit_transform(x_)

print(x_.shape)
print(y.shape)
print(svd.components_.shape)
print((svd.components_[0,:] ** 2).sum())

(30091, 9774)
(30091, 100)
(100, 9774)
0.9999999999999986


## Three components

아래 함수는 scikit-learn TruncatedSVD 의 내부 부분을 옮겨온 것입니다. randomized_svd 는 sparse matrix 에 대하여 효율적인 SVD 연산을 도와주는 함수입니다. 그 결과로 S, Sigma, VT 가 출력됩니다.

In [4]:
from sklearn.utils import check_random_state
from sklearn.utils.extmath import randomized_svd

def fit_svd(X, n_components, n_iter=5, random_state=None):

    if (random_state == None) or isinstance(random_state, int):
        random_state = check_random_state(random_state)

    n_features = X.shape[1]

    if n_components >= n_features:
        raise ValueError("n_components must be < n_features;"
                         " got %d >= %d" % (n_components, n_features))

    U, Sigma, VT = randomized_svd(
        X, n_components,
        n_iter = n_iter,
        random_state = random_state)

    return U, Sigma, VT

U, Sigma, VT = fit_svd(x_, n_components=100)

때로는 Sigma 를 두 부분으로 나눠서 dimension reduction 의 결과와 components 를 만들기도 합니다. components 의 각 축에 중요도를 부여하기 위해서입니다.

In [5]:
print(U.shape)
print(Sigma.shape)
print(VT.shape)

(30091, 100)
(100,)
(100, 9774)


components 의 각 축의 L2 norm 크기는 더 이상 1 이 아닙니다. 0, 1, 2 순으로 각 축의 L2 norm 이 커짐을 확인할 수 있습니다.

In [9]:
S_ = Sigma ** (0.5)

y = U * S_
components = (VT.T * S_).T

print(y.shape)
print(components.shape)
print((components[0,:] ** 2).sum())
print((components[1,:] ** 2).sum())
print((components[2,:] ** 2).sum())

(30091, 100)
(100, 9774)
49.83335905316896
41.272542240242544
30.456088260664192
